In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

db = 'chinook.db'

def run_query(q):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect(db) as conn:
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

show_tables()


,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


### Selecting New Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|Artist Name|Genre|
|:---------|:-----|
|Regal|Hip-Hop|
|Red Tone|Punk|
|Meteor and the Girls|Pop|
|Slim Jim Bites|Blues|

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

Write a query that returns each genre, with the number of tracks sold in the USA:
- in absolute numbers
- in percentages.

In [19]:
albums_to_purchase='''
WITH usa_tracks_sold AS
(select il.*
FROM invoice_line il
INNER JOIN  invoice i ON i.invoice_id= il.invoice_id
INNER JOIN customer c ON c.customer_id = i.customer_id
WHERE c.country ='USA')

SELECT 
g.name genre,
count(uts.quantity) total_qty,
CAST (count(uts.quantity) AS FLOAT) /
(SELECT COUNT(*) from usa_tracks_sold) percentage_sold
FROM usa_tracks_sold uts
LEFT JOIN track t ON t.track_id=uts.track_id
INNER JOIN genre g ON g.genre_id=t.genre_id
group by g.name
order by 2 desc
'''
run_query(albums_to_purchase)

,genre,total_qty,percentage_sold
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


Looks like Rock genre is the the top-seller with 53% of the total tracks sold in USA.

However, since we are choosing among 4 bands, the Blues, Pop and Hip-Hop bands might do well based on sales record.

### Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.

In [3]:
employee_sales_performance = '''
SELECT 
e.first_name||' '||e.last_name employee_name,
e.hire_date,
sum(i.total) total_sales
FROM employee e
LEFT JOIN customer c ON c.support_rep_id = e.employee_id
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1;
'''
run_query(employee_sales_performance)

,employee_name,hire_date,total_sales
0,Jane Peacock,2017-04-01 00:00:00,1731.51
1,Margaret Park,2017-05-03 00:00:00,1584.00
2,Steve Johnson,2017-10-17 00:00:00,1393.92


While Jane Peacock has the most sales, we have to account for the length of time that they've been working. Looks like Steve Johnson has the shortest time with the company already bringing thousands.

### Analyzing Sales by Country

Calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis.

If there is a particular value that you would like to force to the top or bottom of results, you can put what would normally be your most outer query in a subquery with a case statement that adds a numeric column, and then in the outer query sort by that column.

In [41]:
sales_by_country='''
WITH country_other AS
(SELECT
        CASE WHEN
            (SELECT COUNT(*)
            FROM customer
            where country=c.country)=1 
                THEN 'Other'
                ELSE c.country
            END as country,
            c.customer_id,
            il.*
    FROM invoice i
    INNER JOIN customer c ON c.customer_id = i.customer_id
    INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id)

SELECT
country,customers,
total_sales,
avg_sales_per_cust,
avg_order

FROM
(
    SELECT 
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) avg_sales_per_cust,
        SUM(unit_price) / count(distinct invoice_id) avg_order,
        CASE WHEN
        country="Other"
        THEN 1
        ELSE 0
        END as sort
    FROM country_other co
    GROUP BY 1
    ORDER BY sort ASC, 2 DESC);
'''

run_query(sales_by_country)


,country,customers,total_sales,avg_sales_per_cust,avg_order
0,USA,13,1040.49,80.037692,7.942672
1,Canada,8,535.59,66.948750,7.047237
2,Brazil,5,427.68,85.536000,7.011148
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,United Kingdom,3,245.52,81.840000,8.768571
6,Czech Republic,2,273.24,136.620000,9.108000
7,India,2,183.15,91.575000,8.721429
8,Portugal,2,185.13,92.565000,6.383793
9,Other,15,1094.94,72.996000,7.448571


North American countries USA and Canada hold top spots for total sales.

It is challenging to predict which countries  have potential for growth due to limited sample.

However, customers from Czech Republic, Portugal and India seem to have the highest average value of sales per order. 

### Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent you from getting a 100% accurate answer to your question. In this instance, we have two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.



In [48]:
album='''
SELECT a.album_id,
a.title,
t.track_id,
t.name
FROM album a
INNER JOIN track t ON t.album_id = a.album_id
sorty
'''

run_query(album)

,album_id,title,track_id,name
0,1,For Those About To Rock We Salute You,1,For Those About To Rock (We Salute You)
1,1,For Those About To Rock We Salute You,6,Put The Finger On You
2,1,For Those About To Rock We Salute You,7,Let's Get It Up
3,1,For Those About To Rock We Salute You,8,Inject The Venom
4,1,For Those About To Rock We Salute You,9,Snowballed
5,1,For Those About To Rock We Salute You,10,Evil Walks
6,1,For Those About To Rock We Salute You,11,C.O.D.
7,1,For Those About To Rock We Salute You,12,Breaking The Rules
8,1,For Those About To Rock We Salute You,13,Night Of The Long Knives
9,1,For Those About To Rock We Salute You,14,Spellbound


In [49]:
albums_vs_tracks = '''
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;
'''

run_query(albums_vs_tracks)

,album_purchase,number_of_invoices,percent
0,no,500,0.814332
1,yes,114,0.185668


Album purchases account for almost 20% of the total number of individual sales and is therefore a huge chunk of the revenue stream of the company. I recommend that album purchase be retained to keep customers who prefer to buy albums than collection of tracks.